I am indebted to the following resources:
    https://github.com/python-visualization/folium/blob/master/examples/ImageOverlay.ipynb and
    https://www.researchgate.net/post/How_to_plot_a_raster_datageotiff_in_Folium_for_interactive_maps
        
I am always looking for slick ways to present image processing in a simply manner and these helped me with the code below


Start off by converting the raw tif file to a jpg
use
gdal_translate -of JPEG -scale -co worldfile=yes input.tiff output.jpg


In [3]:
import cv2
img = cv2.imread('D:/cv_leaflet/band2_mercator.jpg')
edges = cv2.Canny(img,50,50)
edges = cv2.imwrite('D:/cv_leaflet/edges.jpg', edges)


Lets keep the project associated with the image


In [4]:
import shutil
src = "D:/cv_leaflet/band2_mercator.wld"
dst = "D:/cv_leaflet/edges.wld"
shutil.copy(src,dst) #copy file

'D:/cv_leaflet/edges.wld'

In order to send the newly created images to leaflet we can use a library called folium

In [ ]:
import folium
from folium.plugins import ImageOverlay
import os
from osgeo import gdal
edge_lyr = ('D:/cv_leaflet/edges.jpg')
raw_lyr = ('D:/cv_leaflet/band2_mercator.jpg')

ds = gdal.Open(raw_lyr) 
if ds is None:
    print('Could not open')

#cols and rows
geotransform = ds.GetGeoTransform()
cols = ds.RasterXSize 
rows = ds.RasterYSize 

#Get the extent
xmin=geotransform[0]
ymax=geotransform[3]
xmax=xmin+cols*geotransform[1]
ymin=ymax+rows*geotransform[5]


#m = folium.Map([40, 7], zoom_start=8, tiles='stamentoner')
## tell folium where to load (location) and what backgronud map to use
m = folium.Map([40, 7], zoom_start=8, tiles='stamenterrain')

bounds = [[ymin, xmin], [ymax, xmax]]
print (bounds)

## add my edge detection layer
folium.plugins.ImageOverlay(
    name='My edge detection layer',
    image=edge_lyr,
    bounds=[[ymin, xmin], [ymax, xmax]], ## se above
    opacity=1,
).add_to(m)


## add the raw satellite image below it
folium.plugins.ImageOverlay(
    name='raw satellite',
    image=raw_lyr,
    bounds=[[ymin, xmin], [ymax, xmax]],### its the same image dimensions... be careful
    opacity=1,
).add_to(m)

folium.LayerControl().add_to(m)
## save the map
m.save('D:/cv_leaflet/Computer_vision_leaflet_eg.html')

## display it in the notebook!
m